In [8]:

import os
import json
from PIL import Image
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [9]:
with open('./radar/datasets/anno/train.json', 'r') as file:
    data = json.load(file)

In [10]:
data.keys()

dict_keys(['images', 'annotations', 'categories'])

In [19]:
##label 이상치 확인 -> albumentation에서 에러발생
for info in data["annotations"]:
    bbox = info["bbox"]
    if bbox[0]<0 or bbox[1]<0 or bbox[0]+bbox[2]>640 or bbox[1]+bbox[3] >480:
        print(info)
   
# img_scale = (640, 480)


{'id': 33429, 'image_id': 'train_4620', 'bbox': [622, 246, 20, 47], 'area': 940, 'category_id': 7, 'segmentation': [], 'iscrowd': 0}
{'id': 33845, 'image_id': 'train_4688', 'bbox': [617, 256, 25, 25], 'area': 625, 'category_id': 7, 'segmentation': [], 'iscrowd': 0}
{'id': 33846, 'image_id': 'train_4688', 'bbox': [-1, 196, 68, 90], 'area': 6120, 'category_id': 1, 'segmentation': [], 'iscrowd': 0}
{'id': 42896, 'image_id': 'train_5912', 'bbox': [-1, 101, 59, 182], 'area': 10738, 'category_id': 0, 'segmentation': [], 'iscrowd': 0}


In [19]:
### class 분포확인

label_num={0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for data_info in data["annotations"]:
    label_num[data_info["category_id"]]+=1
    

In [20]:
label_num

{0: 13684, 1: 27754, 2: 2341, 3: 1232, 4: 427, 5: 1172, 6: 29, 7: 3708}

In [2]:

train_dir_prefix = "./radar/datasets/train/"
images = os.listdir(train_dir_prefix)


In [4]:
len(images)

7019

In [5]:
all_pixels = []

for idx,image in enumerate(images):
    if idx % 10 == 0:
        image_path = os.path.join(train_dir_prefix,image)
        image = Image.open(image_path).convert('L')
        image_array = np.array(image)
        all_pixels.append(image_array.flatten())
all_pixels = np.hstack(all_pixels)

# 평균 및 표준 편차 계산
mean = all_pixels.mean()
std = all_pixels.std()
    

In [6]:
mean

139.4229080324816

In [7]:
std

56.34895042201581

In [3]:
from mmdet.apis import DetInferencer

# Choose to use a config
# model_name = 'rtmdet_tiny_8xb32-300e_coco'
model_name = './projects/CO-DETR/configs/codino/co_dino_5scale_swin_l_16xb1_16e_custom.py'

# Setup a checkpoint file to load
# checkpoint = './checkpoints/co_dino_5scale_swin_large_16e_o365tococo-614254c9.pth'
checkpoint = "./work_dirs/co_dino_5scale_swin_l_16xb1_16e_custom/epoch_5.pth"

# Set the device to be used for evaluation
device = 'cuda:0'

# Initialize the DetInferencer
inferencer = DetInferencer(model_name, checkpoint, device)

# Use the detector to do inference
img = './demo/demo.jpg'
result = inferencer(img, out_dir='./output')

Loads checkpoint by local backend from path: ./work_dirs/co_dino_5scale_swin_l_16xb1_16e_custom/epoch_5.pth
08/29 13:16:27 - mmengine - INFO - 
rpn_conv.weight - torch.Size([256, 256, 3, 3]): 
NormalInit: mean=0, std=0.01, bias=0 
 
08/29 13:16:27 - mmengine - INFO - 
rpn_conv.bias - torch.Size([256]): 
NormalInit: mean=0, std=0.01, bias=0 
 
08/29 13:16:27 - mmengine - INFO - 
rpn_cls.weight - torch.Size([9, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
08/29 13:16:27 - mmengine - INFO - 
rpn_cls.bias - torch.Size([9]): 
NormalInit: mean=0, std=0.01, bias=0 
 
08/29 13:16:27 - mmengine - INFO - 
rpn_reg.weight - torch.Size([36, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
08/29 13:16:27 - mmengine - INFO - 
rpn_reg.bias - torch.Size([36]): 
NormalInit: mean=0, std=0.01, bias=0 
 
08/29 13:16:28 - mmengine - INFO - 
bbox_head.fc_cls.weight - torch.Size([81, 1024]): 
NormalInit: mean=0, std=0.01, bias=0 
 
08/29 13:16:28 - mmengine - INFO - 
bbox_head.fc_cls.bias - torc

/mnt/nas2/users/eslim/workspace/comp/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


08/29 13:16:32 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
08/29 13:16:32 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.


Output()

/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be 
removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This 
results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use 
torch.div(a, b, rounding_mode='floor'). (Triggered internally at  
/opt/conda/conda-bld/pytorch_1623448265233/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)

In [4]:
result.keys()
len(result['predictions'])
result['predictions'][0].keys()
result['predictions'][0]["bboxes"]
result['predictions'][0]["scores"]
result['predictions'][0]["labels"]
result['predictions'][0]["bboxes"][0]

[295.40911865234375,
 115.19164276123047,
 378.32366943359375,
 149.26853942871094]

In [7]:
img = '../radar/datasets/test/test_open_0.png'
result = inferencer(img, out_dir='./output')

Output()